# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.2 on the current Colab Runtime...
2024-03-25 00:14:38 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.2-linux-x86_64.tar.gz [135859273/135859273] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.8

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [ ]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2


# Initial Solver
Run Installation First


In [ ]:
import Pkg; Pkg.add("CSV")
import Pkg; Pkg.add("Tables")
import Pkg; Pkg.add("JuMP")
import Pkg; Pkg.add("HiGHS")

import CSV
import Tables
import JuMP
import HiGHS
import JuMP.Containers
using CSV
using Tables
using JuMP
using HiGHS
using JuMP.Containers

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed TableTraits ───────────────── v1.0.1
   Installed PooledArrays ──────────────── v1.4.3
   Installed IteratorInterfaceExtensions ─ v1.0.0
   Installed SentinelArrays ────────────── v1.4.1
   Installed DataAPI ───────────────────── v1.16.0
   Installed Tables ────────────────────── v1.11.1
   Installed InlineStrings ─────────────── v1.4.0
   Installed DataValueInterfaces ───────── v1.0.0
   Installed WorkerUtilities ───────────── v1.6.1
   Installed WeakRefStrings ────────────── v1.4.2
   Installed OrderedCollections ────────── v1.6.3
   Installed Compat ────────────────────── v4.14.0
   Installed FilePathsBase ─────────────── v0.9.21
   Installed TranscodingStreams ────────── v0.10.6
   Installed CodecZlib ─────────────────── v0.7.4
   Installed CSV ───────────────────────── v0.10.13
    Updating `~/.julia/environments/v1.8/Project.toml`
  [336ed68f] + CSV v0.10.13
    Updating `~/.

In [ ]:
input_evacuee_matrix = CSV.File("input_evacuee_matrix.csv") |> Tables.matrix
input_shelter_matrix = CSV.File("input_shelter_matrix.csv") |> Tables.matrix

evacuee_names = input_evacuee_matrix[1:990,1]
shelter_names = input_shelter_matrix[:,1]

187-element Vector{Any}:
 "桜丘小学校"
 "中山中学校"
 "中山小学校"
 "北仙台中学校"
 "台原小学校"
 "旭丘小学校"
 "荒巻小学校"
 "国見小学校"
 "三条中学校"
 "通町小学校"
 "小松島小学校"
 "八幡小学校"
 "木町通小学校"
 ⋮
 "将監東中学校"
 "将監中学校"
 "将監西小学校"
 "高森東小学校"
 "長命ケ丘小学校"
 "南中山中学校"
 "寺岡小学校"
 "住吉台中学校"
 "根白石中学校"
 "館中学校"
 "仙台商業高等学校"
 "市名坂小学校"

In [ ]:
full_cost_matrix = CSV.File("final_cost_matrix.csv") |> Tables.matrix

edited_cost_matrix = full_cost_matrix[:,2:end]

full_cost_array = Containers.DenseAxisArray(edited_cost_matrix, 1:990, 1:187)

shelter_numbers = input_shelter_matrix[1:end,2]

evacuee_numbers = input_evacuee_matrix[1:end,2]

function solve_transportation_problem(data::Containers.DenseAxisArray,shelter_numbers,evacuee_numbers)
    # Get the set of supplies and demands
    O, D = axes(data)
    # Drop the EVACUEES and VACANCIES nodes from our sets
    #O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, x[o in O, d in D] >= 0)
    # Remove arcs with "." cost by fixing them to 0.0.
    for o in O, d in D
        if data[o, d] == "."
            fix(x[o, d], 0.0; force = true)
        end
    end
    @objective(
        model,
        Min,
        sum(data[o, d] * x[o, d] for o in O, d in D if data[o, d] != "."),
    )
    @constraint(model, [o in O], sum(x[o, :]) == evacuee_numbers[o])
    @constraint(model, [d in D], sum(x[:, d]) <= shelter_numbers[d])
    optimize!(model)


    # Construct the solution matrix
    solution_matrix = zeros(length(O), length(D))  # Initialize with zeros

    for (i, o) in enumerate(O)
        for (j, d) in enumerate(D)
            solution_matrix[i, j] = value(x[o, d])
        end
    end

    # Pretty print the solution in the format of the input
    print("    ", join(lpad.(D, 7, ' ')))
    for o in O
        print("\n", o)
        for d in D
            if isapprox(value(x[o, d]), 0.0; atol = 1e-6)
                print("      .")
            else
                print(" ", lpad(value(x[o, d]), 6, ' '))
            end
        end
    end
    return solution_matrix
end

print("\nSolution:\n")
solution_matrix = solve_transportation_problem(full_cost_array, shelter_numbers, evacuee_numbers)
println(solution_matrix)

CSV.write("sendai_solution_matrix.csv", Tables.table(solution_matrix))



Solution:
          1      2      3      4      5      6      7      8      9     10     11     12     13     14     15     16     17     18     19     20     21     22     23     24     25     26     27     28     29     30     31     32     33     34     35     36     37     38     39     40     41     42     43     44     45     46     47     48     49     50     51     52     53     54     55     56     57     58     59     60     61     62     63     64     65     66     67     68     69     70     71     72     73     74     75     76     77     78     79     80     81     82     83     84     85     86     87     88     89     90     91     92     93     94     95     96     97     98     99    100    101    102    103    104    105    106    107    108    109    110    111    112    113    114    115    116    117    118    119    120    121    122    123    124    125    126    127    128    129    130    131    132    133    134    135    136    137    138    139    140    1

Excessive output truncated after 524298 bytes.


397      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .  577.0      .      .      .      .      .      .      .      .      .      .      .      .  694.0      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .  

"sendai_solution_matrix.csv"

In [ ]:
#NOT DONE Total Cost Calculation
solution_cost = full_cost_array.data[1:end-1,1:end-1]
matching_indicies = findall(x-> x == ".", solution_cost)
solution_cost[matching_indicies] .= 0
#solution_matrix .* solution_cost
total_cost_matrix = solution_cost.*solution_matrix
running_total = sum(total_cost_matrix)
initial_running_total = running_total

In [ ]:
print(solution_matrix)

# Testing

In [ ]:
function is_evacuee_numbers_matching(solution_matrix,evacuee_numbers)
  column_wise_solution_sum = sum(solution_matrix, dims=2)
  int_column_wise_solution_sum = Int.(column_wise_solution_sum)
  evacuees_comparison = int_column_wise_solution_sum .== evacuee_numbers
  return prod(evacuees_comparison)
end

is_evacuee_numbers_matching(solution_matrix,evacuee_numbers)

In [ ]:
function is_shelter_not_overflowing(solution_matrix,shelter_numbers)
  row_wise_solution_sum = sum(solution_matrix, dims=1)
  int_row_wise_solution_sum = Int.(row_wise_solution_sum)
  shelter_comparison = int_row_wise_solution_sum[1,:] .<= shelter_numbers
  return prod(shelter_comparison)
end

is_shelter_not_overflowing(solution_matrix,shelter_numbers)

In [ ]:
#Evacuee = shelter_max +1 overload check
println(sum(shelter_numbers))
println(sum(evacuee_numbers))

shelter_evacuee_difference = sum(shelter_numbers) - sum(evacuee_numbers)
overload_evacuee_numbers = deepcopy(evacuee_numbers)
overload_evacuee_numbers[1] += shelter_evacuee_difference

println(sum(overload_evacuee_numbers))

overload_solution_matrix = solve_transportation_problem(full_cost_array, shelter_numbers, overload_evacuee_numbers)
println(overload_solution_matrix)
